In [ ]:
import pandas as pd
df = pd.read_csv('../results/mai_doc2vec_sim.csv')
df

In [ ]:
import spacy
nlp = spacy.load('ja_ginza')
for p in nlp.pipeline:
    print(p)

In [ ]:
import textdistance
# text_pair = df.sort_values('SIM', ascending=False).iloc[203][['T2_Mai', 'T2_Maisho', 'SIM']].values.flatten()
text_pair = df.sample(1)[['T2_Mai', 'T2_Maisho', 'SIM']].values.flatten()
print(text_pair)
print('longest common subsequence：', textdistance.lcsseq(text_pair[0], text_pair[1]))
print('distance: ', textdistance.lcsseq.distance(text_pair[0], text_pair[1]))
print('normalized distance: ', textdistance.lcsseq.normalized_distance(text_pair[0], text_pair[1]))
print('similarity: ', textdistance.lcsseq.similarity(text_pair[0], text_pair[1]))
print('normalized similarity: ', textdistance.lcsseq.normalized_similarity(text_pair[0], text_pair[1]))

In [ ]:
for i in range(2):
    print('-------------------------------------------------')
    doc = nlp(text_pair[i])
    for s in doc.sents:
        print(s)

    from spacy import displacy
    displacy.render(doc, style="ent")

In [ ]:
import stanfordnlp
nlp = stanfordnlp.Pipeline(lang="ja", treebank="ja_gsd")
for i in range(2):
    print('-------------------------------------------------')
    doc = nlp(text_pair[i])
    for sen in doc.sentences:
        print(' '.join(word.text for word in sen.words))

In [ ]:
%%time
import numpy as np
import textdistance
@np.vectorize
def lcs(t1, t2):
    return textdistance.lcsseq.normalized_similarity(t1, t2)

df['lcs_sim'] = lcs(df['T2_Mai'], df['T2_Maisho'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
df_annotated = pd.read_csv('../results/mai_annotated.csv')
df_d2v = df.merge(df_annotated[['ID_Maisho', 'ID_Mai', 'Match']], on=['ID_Maisho', 'ID_Mai'], how='left')
df_d2v['Match'] = df_d2v['Match'].fillna(-1).astype(int)
sns.scatterplot(data=df_d2v, x='SIM', y='lcs_sim', hue='Match', alpha=0.5).set(xlim=(0.0, 1.0), ylim=(0.0, 1.0), aspect=1)
df_d2v.plot.hexbin(x='SIM', y='lcs_sim', gridsize=25, sharex=False).set(xlim=(0.0, 1.0), ylim=(0.0, 1.0), aspect=1)
plt.show()


In [ ]:
lsi = pd.read_csv('../results/mai_lsi_sim.csv')
lsi

In [ ]:
%%time
lsi['lcs_sim'] = lcs(lsi['T2_Mai'], lsi['T2_Maisho'])

In [ ]:
df_lsi = lsi.merge(df_annotated[['ID_Maisho', 'ID_Mai', 'Match']], on=['ID_Maisho', 'ID_Mai'], how='left')
df_lsi['Match'] = df_lsi['Match'].fillna(-1).astype(int)
sns.scatterplot(data=df_lsi, x='SIM', y='lcs_sim', hue='Match', alpha=0.5).set(xlim=(0.0, 1.0), ylim=(0.0, 1.0), aspect=1)
df_lsi.plot.hexbin(x='SIM', y='lcs_sim', gridsize=25, sharex=False).set(xlim=(0.0, 1.0), ylim=(0.0, 1.0), aspect=1)
plt.show()